# BruteForce Approach to determine Frequent Item Sets

In [ ]:
#Import libraries
from timeit import default_timer as timer
from itertools import combinations
from collections import Counter
from itertools import chain
import pandas as pd
import numpy as np
import csv

#Final frequent sets declaration
bruteforce_freq_item = []
count_list = []

#Read from CSV file into DataFrame
all_columns = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15"]
df = pd.read_csv("Z:\Big Data and Data Mining\Assignments\CourseWork1\GroceryStore.csv",names = all_columns)
df.head()

In [ ]:
#open file to write the intermediate updates
output_file = open("Z:\Big Data and Data Mining\Assignments\CourseWork1\BruteForce.txt",'w')

In [ ]:
#Read minimum support and confidence
min_support = input("Enter the value of minimum support\n")
min_confidence = input("Enter the value of minimum confidence\n")

In [ ]:
#Find unique_items in list
def unique_items(dataframe):
    item_list = dataframe.values.tolist()
    flat_item_list = list(chain.from_iterable(item_list))
    unique_item_list = list(set(flat_item_list))
    unique_item_list = list(filter(None, unique_item_list))
    return unique_item_list 

In [ ]:
#Function to write list into file and print it on screen
def write_list_to_file(message,item_list):
    item_list_str = "\n".join(item_list)
    output_file.write(message + item_list_str)
    print(message + item_list_str)

In [ ]:
#Finding support count of combinations
def find_support_count(candidates,item_list):
    count = 0
    k = 0
    support_set = [0]*len(candidates)
    for i in range(len(candidates)):
        for j in range(len(df)):
                if(set(candidates[i]).issubset(set(item_list[j]))): 
                         count += 1
        support_set[k] = count
        count = 0
        k = k+1
    return support_set

In [ ]:
#Prune the sets that donot satisfy min support value
def prune(candidates,count):
    freq_list = []
    count_freq = []
    for i in range(0,len(candidates)):
        if(count[i]>=int(min_support)):
            freq_list.append(candidates[i])
            count_freq.append(count[i])
    return freq_list,count_freq

In [ ]:
#Bruteforce approach to find frequent item set
def bruteforce(dataframe,n):
    start = timer()
#   Find unique items
    unique_item_list = unique_items(dataframe)
    item_list = [x for x in unique_item_list if pd.isnull(x) == False]
    data_list = df.values.tolist() 
    
# Find combinations, candidates and then prune to get final frequent set
    candidate_list = []
    for i in range(1,n):
        comb = combinations(item_list,i)
        candidates = list(comb)
        candidate_list.append(candidates)
        count = find_support_count(candidates,data_list)
        freq_item_set,count = prune(candidates,count) 
        if(count):
            count_list.append(count)
        if(freq_item_set):
            bruteforce_freq_item.append(freq_item_set)
    end = timer()
    
    #prepare to write to output file
    candidate_str = [0]*len(candidate_list)
    bruteforce_str = [0]*len(bruteforce_freq_item)
    for i in range(len(candidate_list)):
        candidate_str[i] = ''.join(str(candidate_list[i]))
    for i in range(len(bruteforce_freq_item)):
        bruteforce_str[i] = ''.join(str(bruteforce_freq_item[i]))
    
    #calculate time taken to execute for bruteforce approach
    total_time = end - start
    
    
    #Write output to file and print the output
    output_file.write("1.minimum support :" + min_support)
    print("1.minimum support:"+min_support)
    output_file.write("\n\n2.minimum confidence :" + min_confidence)
    print("\n\n2.minimum confidence :" + min_confidence)
    output_file.write("\n\n3.Brute Force Approach:")  
    print("\n\n3.Brute Force Approach:")
    write_list_to_file("\n\n\n3a.Unique item list:\n",item_list)
    write_list_to_file("\n\n\n3b.1,2,3,4,5 item sets:\n",candidate_str)
    write_list_to_file("\n\n\n3c.frequent item sets:\n",bruteforce_str)
    output_file.write("\n\n4.Time taken for execution using bruteforce approach:"+str(total_time)) 
    print("\n\n4.Time taken for execution using bruteforce approach:" + str(total_time))

In [ ]:
#Bruteforce function call
bruteforce(df,6)

In [ ]:
#Function call Association rules
def association_rules():
    confidence_percentage = int(min_confidence)/100
    output_file.write("\n\n5.Association Rules:\n")
    n = len(bruteforce_freq_item)-1
    x = n
    z = n
    for i in range(n,0,-1):
        for j in range(len(bruteforce_freq_item[z])):
            numerator = bruteforce_freq_item[z][j]
            numerator_value = count_list[z][j]
            x = z
            for y in range(0,z):
                for k in range(len(count_list[x-1])):
                    denominator = bruteforce_freq_item[x-1][k]
                    denominator_value = count_list[x-1][k]
                    if(denominator_value):
                        if(set(denominator).issubset(set(numerator))):
                            if((numerator_value/denominator_value)>=confidence_percentage):
                                print(str(denominator)+"----->"+str(set(numerator)-set(denominator))+"             :"+str(round((numerator_value/denominator_value)*100,2))+"%")
                                output_file.write("\n"+str(denominator)+"----->"+str(set(numerator)-set(denominator))+"             :"+str(round((numerator_value/denominator_value)*100,2))+"%")
                x = x-1
        z = z-1

In [ ]:
#association rule function call
association_rules()

In [ ]:
#Close output file after writing
output_file.close()